[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/Semana%2009_Fundamentos%20NLP/Script_Clase18_FundamentosNLP_y_Embedings.ipynb)

# 🤖 Inteligencia Artificial Aplicada para la Economía
## Universidad de los Andes

### 👨‍🏫 Profesores
- **Profesor Magistral:** Camilo Vega Barbosa
- **Asistente de Docencia:** Sergio Julian Zona Moreno

### 📚 Fundamentos NLP: Similitud Semántica entre Productos y Categorías
Este notebook demuestra la implementación práctica de conceptos fundamentales de Procesamiento de Lenguaje Natural (NLP):

1. **Tokenización ✂️**
   - División del texto en unidades básicas
   - Procesamiento a nivel de palabras
   - Manejo de términos compuestos

2. **Normalización 🧹**
   - Conversión a minúsculas
   - Lematización
   - Eliminación de stopwords y puntuación

3. **Vectorización 🔢**
   - Word Embeddings pre-entrenados
   - Representación vectorial de palabras
   - Espacio semántico multidimensional

4. **Similitud de Coseno 📐**
   - Cálculo matemático paso a paso
   - Implementación optimizada con NumPy
   - Interpretación de resultados
   - Visualización de relaciones semánticas

### 🎯 Objetivo
Identificar relaciones semánticas entre productos de supermercado y sus categorías utilizando:
- Embeddings pre-entrenados de SpaCy
- Cálculo de similitud de coseno
- Clasificación automática basada en similitud semántica

### 🔍 Aplicaciones Prácticas:
- Sistemas de búsqueda semántica en e-commerce
- Categorización automática de productos
- Recomendaciones basadas en similitud
- Análisis de mercado y agrupamiento

### Requisitos Técnicos:
- **Entorno de Ejecución**: Google Colab o Jupyter Notebook
- **Bibliotecas Necesarias**:
  - spaCy (con modelo español: es_core_news_md)
  - NumPy
  - scikit-learn
  - Matplotlib
  - Seaborn
- **Memoria RAM**: Mínimo 4GB recomendados
- **Tiempo Estimado**: 15-20 minutos para ejecución completa



## Ejemplo de Embeddings para productos de supermercado

In [ ]:
# Instalar las bibliotecas principales
!pip install spacy scikit-learn matplotlib seaborn numpy -q

# Descargar los modelos de lenguaje de spaCy para español y alemán
!python -m spacy download es_core_news_md -q
!python -m spacy download en_core_news_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 18.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

✘ No compatible package found for 'en_core_news_md' (spaCy v3.8.4)



In [ ]:
"""
EJERCICIO DE SIMILITUD SEMÁNTICA: PRODUCTOS Y CATEGORÍAS DE SUPERMERCADO 🛒

Este script demuestra el proceso completo de NLP comparando productos de supermercado
con sus posibles categorías, mostrando cómo los embeddings capturan relaciones semánticas
entre palabras del mismo idioma.
"""

# ====================== IMPORTACIÓN DE BIBLIOTECAS ======================
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA  # Añadido para PCA

# ====================== DEFINICIÓN DE FUNCIONES ======================

def procesar_palabra(palabra, nlp, mostrar_detalles=True):
    """
    Realiza el proceso completo de NLP en una palabra:
    1. Tokenización ✂️
    2. Normalización 🧹
    3. Vectorización 🔢
    """
    # PASO 1: TOKENIZACIÓN ✂️
    # Procesamos la palabra con SpaCy
    doc = nlp(palabra.lower())  # Convertimos a minúsculas para mejorar resultados

    if mostrar_detalles:
        print(f"\n🔍 Analizando: '{palabra}'")

        # Mostramos los tokens
        print(f"  ✂️ Tokenización:")
        for token in doc:
            print(f"    - Token: '{token.text}'")

        # PASO 2: NORMALIZACIÓN 🧹
        print(f"  🧹 Normalización:")
        for token in doc:
            print(f"    - Original: '{token.text}'")
            print(f"    - Minúsculas: '{token.text.lower()}'")
            print(f"    - Lema: '{token.lemma_}'")

        # PASO 3: VECTORIZACIÓN 🔢
        print(f"  🔢 Vectorización:")
        print(f"    - Dimensiones del vector: {doc.vector.shape}")

        # Mostramos una pequeña muestra del vector
        print(f"    - Muestra del vector: {doc.vector[:5]}...")

    # Devolvemos el vector y el documento procesado
    return doc.vector, doc

def calcular_similitud_productos_categorias(productos, categorias, clasificacion, nlp):
    """
    Calcula y visualiza la matriz de similitud entre productos y categorías.

    Args:
        productos: Lista de productos
        categorias: Lista de categorías
        clasificacion: Diccionario que asigna cada producto a su categoría correcta
        nlp: Modelo de spaCy cargado
    """
    # Matrices para almacenar vectores y similitudes
    vectores_productos = []
    vectores_categorias = []
    matriz_similitud = np.zeros((len(productos), len(categorias)))

    # Procesamos cada producto
    print("\n🔄 PROCESANDO PRODUCTOS")
    for producto in productos:
        vector, _ = procesar_palabra(producto, nlp, mostrar_detalles=True)
        vectores_productos.append(vector)

    # Procesamos cada categoría
    print("\n🔄 PROCESANDO CATEGORÍAS")
    for categoria in categorias:
        vector, _ = procesar_palabra(categoria, nlp, mostrar_detalles=True)
        vectores_categorias.append(vector)

    # Calculamos la similitud de coseno entre cada producto y categoría
    print("\n📊 CALCULANDO MATRIZ DE SIMILITUD")
    for i, vec_producto in enumerate(vectores_productos):
        for j, vec_categoria in enumerate(vectores_categorias):
            # Preparamos los vectores para calcular similitud de coseno
            vec_producto_reshaped = vec_producto.reshape(1, -1)
            vec_categoria_reshaped = vec_categoria.reshape(1, -1)

            # FÓRMULA DE SIMILITUD DE COSENO: cos(θ) = (A·B)/(|A|·|B|)
            sim = cosine_similarity(vec_producto_reshaped, vec_categoria_reshaped)[0][0]
            matriz_similitud[i, j] = sim

            # Destacamos si es la categoría correcta
            es_categoria_correcta = categorias[j] == clasificacion[productos[i]]
            destacado = "✓" if es_categoria_correcta else ""

            print(f"  Similitud entre '{productos[i]}' y '{categorias[j]}': {sim:.4f} {destacado}")

    # Visualizamos la matriz de similitud
    visualizar_matriz_similitud(productos, categorias, matriz_similitud, clasificacion)

    # Encontramos las mejores coincidencias
    encontrar_mejores_coincidencias(productos, categorias, matriz_similitud, clasificacion)

    return matriz_similitud, vectores_productos, vectores_categorias

def visualizar_matriz_similitud(productos, categorias, matriz_similitud, clasificacion):
    """
    Visualiza la matriz de similitud entre productos y categorías como un mapa de calor.
    """
    plt.figure(figsize=(12, 8))

    # Creamos el mapa de calor con etiquetas arriba
    ax = sns.heatmap(
        matriz_similitud,
        annot=True,
        fmt=".2f",
        cmap="Blues",
        xticklabels=categorias,
        yticklabels=productos,
        vmin=0,
        vmax=1,
        cbar_kws={'label': 'Similitud de Coseno'}
    )

    # Colocamos las etiquetas de columnas (categorías) en la parte superior
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')

    # Añadimos un marcador para las categorías correctas
    # Creamos un diccionario inverso para encontrar índices
    indices_cat = {cat: i for i, cat in enumerate(categorias)}

    for i, producto in enumerate(productos):
        categoria_correcta = clasificacion[producto]
        j = indices_cat[categoria_correcta]

        # Dibujamos un rectángulo alrededor de la categoría correcta
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False,
                                 edgecolor='red', lw=2, clip_on=False))

    plt.title("Similitud de Coseno entre Productos y Categorías", fontsize=14, pad=20)
    plt.xlabel("Categorías", fontsize=12)
    plt.ylabel("Productos", fontsize=12)
    plt.tight_layout()
    plt.show()

def encontrar_mejores_coincidencias(productos, categorias, matriz_similitud, clasificacion):
    """
    Encuentra y muestra las mejores coincidencias entre productos y categorías.
    """
    print("\n🏆 MEJORES COINCIDENCIAS POR PRODUCTO")

    # Contamos cuántas veces acertamos la categoría correcta
    aciertos = 0

    for i, producto in enumerate(productos):
        # Encontramos la categoría con mayor similitud
        mejor_indice = np.argmax(matriz_similitud[i])
        mejor_similitud = matriz_similitud[i, mejor_indice]
        mejor_categoria = categorias[mejor_indice]

        # Verificamos si es la categoría correcta
        categoria_correcta = clasificacion[producto]
        es_correcta = mejor_categoria == categoria_correcta

        if es_correcta:
            aciertos += 1
            resultado = "✅ CORRECTO"
        else:
            resultado = f"❌ INCORRECTO (debería ser '{categoria_correcta}')"

        print(f"  '{producto}' → '{mejor_categoria}' (similitud: {mejor_similitud:.4f}) {resultado}")

        # Interpretamos la similitud
        if mejor_similitud > 0.7:
            interpretacion = "Alta similitud semántica"
        elif mejor_similitud > 0.4:
            interpretacion = "Similitud semántica moderada"
        elif mejor_similitud > 0.2:
            interpretacion = "Baja similitud semántica"
        else:
            interpretacion = "Similitud muy baja o nula"

        print(f"    Interpretación: {interpretacion}")

    # Mostramos la precisión general del modelo
    precision = aciertos / len(productos) * 100
    print(f"\n📈 Precisión del modelo: {precision:.1f}% ({aciertos}/{len(productos)} aciertos)")

def demostrar_frases_vs_palabras(nlp):
    """
    Demuestra cómo los vectores de frases difieren de los vectores de palabras individuales.
    """
    print("\n🧪 DEMOSTRACIÓN: PALABRAS VS FRASES")

    # Definimos nuestras palabras y frases
    palabra = "tomate"
    frase = "tomate fresco"
    categoria1 = "verdura"
    categoria2 = "fruta"

    # Procesamos con SpaCy
    vec_palabra = nlp(palabra.lower()).vector
    vec_frase = nlp(frase.lower()).vector
    vec_cat1 = nlp(categoria1.lower()).vector
    vec_cat2 = nlp(categoria2.lower()).vector

    # Calculamos similitudes
    sim_palabra_cat1 = cosine_similarity(vec_palabra.reshape(1, -1), vec_cat1.reshape(1, -1))[0][0]
    sim_palabra_cat2 = cosine_similarity(vec_palabra.reshape(1, -1), vec_cat2.reshape(1, -1))[0][0]
    sim_frase_cat1 = cosine_similarity(vec_frase.reshape(1, -1), vec_cat1.reshape(1, -1))[0][0]
    sim_frase_cat2 = cosine_similarity(vec_frase.reshape(1, -1), vec_cat2.reshape(1, -1))[0][0]

    print(f"Comparando similitudes:")
    print(f"  • '{palabra}' con '{categoria1}': {sim_palabra_cat1:.4f}")
    print(f"  • '{palabra}' con '{categoria2}': {sim_palabra_cat2:.4f}")
    print(f"  • '{frase}' con '{categoria1}': {sim_frase_cat1:.4f}")
    print(f"  • '{frase}' con '{categoria2}': {sim_frase_cat2:.4f}")

    # Visualizamos las diferencias
    labels = [f"'{palabra}'-'{categoria1}'", f"'{palabra}'-'{categoria2}'",
              f"'{frase}'-'{categoria1}'", f"'{frase}'-'{categoria2}'"]
    valores = [sim_palabra_cat1, sim_palabra_cat2, sim_frase_cat1, sim_frase_cat2]

    plt.figure(figsize=(10, 6))
    plt.bar(labels, valores, color=['blue', 'orange', 'green', 'red'])
    plt.ylim(0, max(valores) * 1.2)
    plt.title("Efecto del Contexto en la Similitud Semántica")
    plt.ylabel("Similitud de Coseno")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print("\n💡 CONCLUSIÓN:")
    print("  Este ejemplo muestra cómo SpaCy considera el contexto completo al generar vectores.")
    print("  La adición de 'fresco' modifica la representación semántica del tomate,")
    print("  posiblemente acercándolo más a la categoría de verdura que de fruta.")

def explicar_similitud_coseno(matriz_similitud, productos, categorias, vectores_productos, vectores_categorias):
    """
    Explica el concepto de similitud de coseno con una visualización usando PCA.
    Usa los valores ya calculados de la matriz de similitud para mantener consistencia.
    """
    print("\n📐 EXPLICACIÓN: SIMILITUD DE COSENO")
    print("La similitud de coseno mide el ángulo entre dos vectores:")
    print("  • Similitud = 1: Los vectores apuntan en la misma dirección (conceptos idénticos)")
    print("  • Similitud = 0: Los vectores son perpendiculares (conceptos no relacionados)")
    print("  • Similitud = -1: Los vectores apuntan en direcciones opuestas (raramente ocurre)")

    # Seleccionamos los productos y categorías específicos para la demostración
    producto_indice = productos.index("manzana")
    categoria_relacionada_indice = categorias.index("frutas")
    categoria_no_relacionada_indice = categorias.index("limpieza")

    palabra1 = productos[producto_indice]
    palabra2 = categorias[categoria_relacionada_indice]
    palabra3 = categorias[categoria_no_relacionada_indice]

    vec1 = vectores_productos[producto_indice]
    vec2 = vectores_categorias[categoria_relacionada_indice]
    vec3 = vectores_categorias[categoria_no_relacionada_indice]

    # Obtenemos las similitudes de la matriz ya calculada para mantener consistencia
    sim_12 = matriz_similitud[producto_indice, categoria_relacionada_indice]
    sim_13 = matriz_similitud[producto_indice, categoria_no_relacionada_indice]

    # Reducir dimensionalidad para visualización (PCA a 2D)
    pca = PCA(n_components=2)

    # Juntar los vectores para aplicar PCA
    vectores_combinados = np.vstack([vec1, vec2, vec3])
    vectores_2d = pca.fit_transform(vectores_combinados)

    # Extraer los vectores reducidos
    vec1_2d = vectores_2d[0]  # Manzana
    vec2_2d = vectores_2d[1]  # Fruta
    vec3_2d = vectores_2d[2]  # Limpieza

    # Crear visualización
    plt.figure(figsize=(8, 6))
    origen = [0, 0]

    # Dibujamos los vectores en 2D
    plt.quiver(*origen, *vec1_2d, angles='xy', scale_units='xy', scale=1, color='blue',
               label=f'{palabra1}')
    plt.quiver(*origen, *vec2_2d, angles='xy', scale_units='xy', scale=1, color='green',
               label=f'{palabra2} - sim:{sim_12:.2f}')
    plt.quiver(*origen, *vec3_2d, angles='xy', scale_units='xy', scale=1, color='red',
               label=f'{palabra3} - sim:{sim_13:.2f}')

    # Configurar gráfico
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    plt.legend(loc='upper left')
    plt.title('Similitud de Coseno: Representación Geométrica (vectores reales)')

    # Calcular ángulos para dibujar arcos
    angulo1 = np.arctan2(vec1_2d[1], vec1_2d[0])
    angulo2 = np.arctan2(vec2_2d[1], vec2_2d[0])
    angulo3 = np.arctan2(vec3_2d[1], vec3_2d[0])

    # Dibujar arcos para mostrar ángulos
    radio = 0.2 * min(np.linalg.norm(vec1_2d), np.linalg.norm(vec2_2d), np.linalg.norm(vec3_2d))

    t1 = np.linspace(min(angulo1, angulo2), max(angulo1, angulo2), 100)
    plt.plot(radio * np.cos(t1), radio * np.sin(t1), 'g-', lw=2, alpha=0.5)

    t2 = np.linspace(min(angulo1, angulo3), max(angulo1, angulo3), 100)
    plt.plot(radio * np.cos(t2), radio * np.sin(t2), 'r-', lw=2, alpha=0.5)

    plt.tight_layout()
    plt.show()

    # Explicación matemática usando los vectores reales pero con los valores ya calculados
    print("\nFÓRMULA MATEMÁTICA:")
    print("  similitud_coseno(A, B) = (A·B) / (|A| × |B|)")
    print("Donde:")
    print("  • A·B es el producto punto de los vectores")
    print("  • |A| y |B| son las magnitudes (normas) de los vectores")
    print("\nEjemplo con valores reales de la matriz:")
    print(f"  • Similitud entre '{palabra1}' y '{palabra2}': {sim_12:.4f}")
    print(f"  • Similitud entre '{palabra1}' y '{palabra3}': {sim_13:.4f}")
    print("\nEsta representación visual usa PCA para reducir la dimensionalidad de los vectores,")
    print("preservando las similitudes calculadas en la matriz anterior.")

# ====================== FUNCIÓN PRINCIPAL ======================

def main():
    """Función principal del programa."""
    print("🛒 EJERCICIO DE SIMILITUD SEMÁNTICA: PRODUCTOS Y CATEGORÍAS 🛒")
    print("="*80)
    print("Este ejercicio demuestra el proceso completo de NLP:")
    print("1. Tokenización ✂️: Dividir el texto en unidades básicas")
    print("2. Normalización 🧹: Estandarizar el texto (minúsculas, lematización)")
    print("3. Vectorización 🔢: Convertir texto en representaciones numéricas")
    print("4. Similitud de coseno 📏: Medir la similitud semántica entre palabras")
    print("="*80)

    # Definimos productos y categorías de supermercado
    productos = [
        "manzana", "detergente", "pollo", "zanahoria",
        "shampoo", "arroz", "atún", "pasta", "tomate",
        "jabón", "lechuga", "cepillo de dientes"
    ]

    categorias = [
        "frutas", "limpieza", "carnes", "verduras",
        "higiene", "cereales", "pescados", "lácteos"
    ]

    # Clasificación correcta de productos (para evaluación)
    clasificacion = {
        "manzana": "frutas",
        "detergente": "limpieza",
        "pollo": "carnes",
        "zanahoria": "verduras",
        "shampoo": "higiene",
        "arroz": "cereales",
        "atún": "pescados",
        "pasta": "cereales",
        "tomate": "verduras",
        "jabón": "higiene",
        "lechuga": "verduras",
        "cepillo de dientes": "higiene"
    }

    # Cargamos el modelo de lenguaje de spaCy
    print("\nCargando modelo de lenguaje...")
    try:
        nlp = spacy.load("es_core_news_md")
        print("✓ Modelo español cargado")
    except OSError:
        print("! Modelo español no encontrado. Instalando...")
        import subprocess
        subprocess.call(["python", "-m", "spacy", "download", "es_core_news_md"])
        nlp = spacy.load("es_core_news_md")

    # Demostramos las diferencias entre vectores de palabras y frases
    demostrar_frases_vs_palabras(nlp)

    # Calculamos la similitud entre productos y categorías
    matriz_similitud, vectores_productos, vectores_categorias = calcular_similitud_productos_categorias(
        productos,
        categorias,
        clasificacion,
        nlp
    )

    # Explicamos la similitud de coseno usando los valores de la matriz ya calculada
    explicar_similitud_coseno(matriz_similitud, productos, categorias, vectores_productos, vectores_categorias)

    print("\n💡 CONCLUSIÓN")
    print("Este ejercicio muestra cómo los modelos de embeddings pueden identificar")
    print("relaciones semánticas entre productos y sus categorías dentro del mismo idioma.")
    print("Observaciones importantes:")
    print("1. La normalización mejora los resultados de similitud")
    print("2. Los embeddings capturan relaciones semánticas incluso cuando no hay similitud léxica")
    print("3. El contexto (palabras adicionales) puede modificar significativamente la representación")
    print("4. Estos conceptos son fundamentales en aplicaciones como:")
    print("   - Sistemas de búsqueda semántica")
    print("   - Categorización automática de productos")
    print("   - Recomendaciones basadas en similitud")
    print("   - Análisis de mercado y agrupamiento")

# ====================== EJECUCIÓN DEL PROGRAMA ======================

if __name__ == "__main__":
    main()

## Ejemplo de Embeddings para productos de supermercado en inglés

In [ ]:
"""
EJERCICIO DE SIMILITUD SEMÁNTICA MULTILINGÜE: PRODUCTOS DE SUPERMERCADO 🛒

Este script demuestra el proceso completo de NLP comparando productos de supermercado
en español e inglés, mostrando cómo los embeddings capturan similitudes semánticas
a través de diferentes idiomas.
"""

# ====================== IMPORTACIÓN DE BIBLIOTECAS ======================
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA  # Añadido para PCA

# ====================== DEFINICIÓN DE FUNCIONES ======================

def procesar_palabra(palabra, nlp, mostrar_detalles=True):
    """
    Realiza el proceso completo de NLP en una palabra:
    1. Tokenización ✂️
    2. Normalización 🧹
    3. Vectorización 🔢
    """
    # PASO 1: TOKENIZACIÓN ✂️
    # Procesamos la palabra con SpaCy
    doc = nlp(palabra.lower())  # Convertimos a minúsculas para mejorar resultados

    if mostrar_detalles:
        print(f"\n🔍 Analizando: '{palabra}'")

        # Mostramos los tokens
        print(f"  ✂️ Tokenización:")
        for token in doc:
            print(f"    - Token: '{token.text}'")

        # PASO 2: NORMALIZACIÓN 🧹
        print(f"  🧹 Normalización:")
        for token in doc:
            print(f"    - Original: '{token.text}'")
            print(f"    - Minúsculas: '{token.text.lower()}'")
            print(f"    - Lema: '{token.lemma_}'")

        # PASO 3: VECTORIZACIÓN 🔢
        print(f"  🔢 Vectorización:")
        print(f"    - Dimensiones del vector: {doc.vector.shape}")

        # Mostramos una pequeña muestra del vector
        print(f"    - Muestra del vector: {doc.vector[:5]}...")

    # Devolvemos el vector y el documento procesado
    return doc.vector, doc

def calcular_similitud_productos(productos_es, productos_en, traducciones, nlp_es, nlp_en):
    """
    Calcula y visualiza la matriz de similitud entre productos en español e inglés.

    Args:
        productos_es: Lista de productos en español
        productos_en: Lista de productos en inglés
        traducciones: Diccionario con las traducciones correctas
        nlp_es: Modelo de spaCy para español
        nlp_en: Modelo de spaCy para inglés
    """
    # Matrices para almacenar vectores y similitudes
    vectores_es = []
    vectores_en = []
    matriz_similitud = np.zeros((len(productos_es), len(productos_en)))

    # Procesamos cada producto en español
    print("\n🔄 PROCESANDO PRODUCTOS EN ESPAÑOL")
    for producto in productos_es:
        vector, _ = procesar_palabra(producto, nlp_es, mostrar_detalles=True)
        vectores_es.append(vector)

    # Procesamos cada producto en inglés
    print("\n🔄 PROCESANDO PRODUCTOS EN INGLÉS")
    for producto in productos_en:
        vector, _ = procesar_palabra(producto, nlp_en, mostrar_detalles=True)
        vectores_en.append(vector)

    # Calculamos la similitud de coseno entre cada par de productos
    print("\n📊 CALCULANDO MATRIZ DE SIMILITUD")
    for i, vec_es in enumerate(vectores_es):
        for j, vec_en in enumerate(vectores_en):
            # Preparamos los vectores para calcular similitud de coseno
            vec_es_reshaped = vec_es.reshape(1, -1)
            vec_en_reshaped = vec_en.reshape(1, -1)

            # FÓRMULA DE SIMILITUD DE COSENO: cos(θ) = (A·B)/(|A|·|B|)
            sim = cosine_similarity(vec_es_reshaped, vec_en_reshaped)[0][0]
            matriz_similitud[i, j] = sim

            # Destacamos si es la traducción correcta
            es_traduccion = productos_en[j] == traducciones[productos_es[i]]
            destacado = "✓" if es_traduccion else ""

            print(f"  Similitud entre '{productos_es[i]}' y '{productos_en[j]}': {sim:.4f} {destacado}")

    # Visualizamos la matriz de similitud
    visualizar_matriz_similitud(productos_es, productos_en, matriz_similitud, traducciones)

    # Encontramos las mejores coincidencias
    encontrar_mejores_coincidencias(productos_es, productos_en, matriz_similitud, traducciones)

    return matriz_similitud, vectores_es, vectores_en

def visualizar_matriz_similitud(productos_es, productos_en, matriz_similitud, traducciones):
    """
    Visualiza la matriz de similitud entre productos como un mapa de calor.
    """
    plt.figure(figsize=(10, 8))

    # Creamos el mapa de calor con etiquetas arriba
    ax = sns.heatmap(
        matriz_similitud,
        annot=True,
        fmt=".2f",
        cmap="Blues",
        xticklabels=productos_en,
        yticklabels=productos_es,
        vmin=-0.1,
        vmax=1,
        cbar_kws={'label': 'Similitud de Coseno'}
    )

    # Colocamos las etiquetas de columnas (productos ingleses) en la parte superior
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')

    # Añadimos un marcador para las traducciones correctas
    # Creamos un diccionario inverso para encontrar índices
    indices_en = {prod: i for i, prod in enumerate(productos_en)}

    for i, prod_es in enumerate(productos_es):
        traduccion = traducciones[prod_es]
        j = indices_en[traduccion]

        # Dibujamos un rectángulo alrededor de la traducción correcta
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False,
                                 edgecolor='red', lw=2, clip_on=False))

    plt.title("Similitud de Coseno entre Productos (Español - Inglés)", fontsize=14, pad=20)
    plt.xlabel("Productos en Inglés", fontsize=12)
    plt.ylabel("Productos en Español", fontsize=12)
    plt.tight_layout()
    plt.show()

def encontrar_mejores_coincidencias(productos_es, productos_en, matriz_similitud, traducciones):
    """
    Encuentra y muestra las mejores coincidencias entre productos.
    """
    print("\n🏆 MEJORES COINCIDENCIAS POR PRODUCTO")

    # Contamos cuántas veces acertamos la traducción correcta
    aciertos = 0

    for i, producto_es in enumerate(productos_es):
        # Encontramos el producto inglés con mayor similitud
        mejor_indice = np.argmax(matriz_similitud[i])
        mejor_similitud = matriz_similitud[i, mejor_indice]
        mejor_producto_en = productos_en[mejor_indice]

        # Verificamos si es la traducción correcta
        traduccion_correcta = traducciones[producto_es]
        es_correcta = mejor_producto_en == traduccion_correcta

        if es_correcta:
            aciertos += 1
            resultado = "✅ CORRECTO"
        else:
            resultado = f"❌ INCORRECTO (debería ser '{traduccion_correcta}')"

        print(f"  '{producto_es}' → '{mejor_producto_en}' (similitud: {mejor_similitud:.4f}) {resultado}")

        # Interpretamos la similitud
        if mejor_similitud > 0.7:
            interpretacion = "Alta similitud semántica"
        elif mejor_similitud > 0.4:
            interpretacion = "Similitud semántica moderada"
        elif mejor_similitud > 0.2:
            interpretacion = "Baja similitud semántica"
        else:
            interpretacion = "Similitud muy baja o nula"

        print(f"    Interpretación: {interpretacion}")

    # Mostramos la precisión general del modelo
    precision = aciertos / len(productos_es) * 100
    print(f"\n📈 Precisión del modelo: {precision:.1f}% ({aciertos}/{len(productos_es)} aciertos)")

def demostrar_mejora_normalizacion(palabra_es, palabra_en, nlp_es, nlp_en):
    """
    Demuestra la importancia de la normalización comparando similitudes.
    """
    print("\n🧪 DEMOSTRACIÓN: IMPORTANCIA DE LA NORMALIZACIÓN")

    # Sin normalización
    doc_es_orig = nlp_es(palabra_es)
    doc_en_orig = nlp_en(palabra_en)

    # Con normalización (minúsculas)
    doc_es_norm = nlp_es(palabra_es.lower())
    doc_en_norm = nlp_en(palabra_en.lower())

    # Calculamos similitudes
    sim_orig = cosine_similarity(
        doc_es_orig.vector.reshape(1, -1),
        doc_en_orig.vector.reshape(1, -1)
    )[0][0]

    sim_norm = cosine_similarity(
        doc_es_norm.vector.reshape(1, -1),
        doc_en_norm.vector.reshape(1, -1)
    )[0][0]

    print(f"Comparando '{palabra_es}' con '{palabra_en}':")
    print(f"  • Similitud SIN normalización: {sim_orig:.4f}")
    print(f"  • Similitud CON normalización: {sim_norm:.4f}")
    print(f"  • Mejora: {(sim_norm - sim_orig):.4f} ({(sim_norm - sim_orig) / max(0.0001, abs(sim_orig)) * 100:.1f}%)")

def explicar_similitud_coseno(matriz_similitud, productos_es, productos_en, vectores_es, vectores_en, traducciones):
    """
    Explica el concepto de similitud de coseno con una visualización usando PCA.
    Usa los valores ya calculados de la matriz de similitud para mantener consistencia.
    """
    print("\n📐 EXPLICACIÓN: SIMILITUD DE COSENO")
    print("La similitud de coseno mide el ángulo entre dos vectores:")
    print("  • Similitud = 1: Los vectores apuntan en la misma dirección (productos idénticos)")
    print("  • Similitud = 0: Los vectores son perpendiculares (productos no relacionados)")
    print("  • Similitud = -1: Los vectores apuntan en direcciones opuestas (raramente ocurre)")

    # Seleccionamos los productos específicos para la demostración
    palabra_es = "pan"  # Producto en español
    palabra_en_relacionada = "bread"  # Traducción correcta
    palabra_en_no_relacionada = "soap"  # Palabra no relacionada

    # Encontramos los índices en nuestras listas
    indice_es = productos_es.index(palabra_es)
    indice_en_rel = productos_en.index(palabra_en_relacionada)
    indice_en_no_rel = productos_en.index(palabra_en_no_relacionada)

    # Extraemos los vectores
    vec1 = vectores_es[indice_es]
    vec2 = vectores_en[indice_en_rel]
    vec3 = vectores_en[indice_en_no_rel]

    # Obtenemos las similitudes de la matriz ya calculada para mantener consistencia
    sim_12 = matriz_similitud[indice_es, indice_en_rel]
    sim_13 = matriz_similitud[indice_es, indice_en_no_rel]

    # Reducir dimensionalidad para visualización (PCA a 2D)
    pca = PCA(n_components=2)

    # Juntar los vectores para aplicar PCA
    vectores_combinados = np.vstack([vec1, vec2, vec3])
    vectores_2d = pca.fit_transform(vectores_combinados)

    # Extraer los vectores reducidos
    vec1_2d = vectores_2d[0]  # Pan
    vec2_2d = vectores_2d[1]  # Bread
    vec3_2d = vectores_2d[2]  # Soap

    # Crear visualización
    plt.figure(figsize=(8, 6))
    origen = [0, 0]

    # Dibujamos los vectores en 2D
    plt.quiver(*origen, *vec1_2d, angles='xy', scale_units='xy', scale=1, color='blue',
               label=f'{palabra_es} (español)')
    plt.quiver(*origen, *vec2_2d, angles='xy', scale_units='xy', scale=1, color='green',
               label=f'{palabra_en_relacionada} (inglés) - sim:{sim_12:.2f}')
    plt.quiver(*origen, *vec3_2d, angles='xy', scale_units='xy', scale=1, color='red',
               label=f'{palabra_en_no_relacionada} (inglés) - sim:{sim_13:.2f}')

    # Configurar gráfico
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    plt.legend(loc='upper left')
    plt.title('Similitud de Coseno: Representación Geométrica (vectores reales)')

    # Calcular ángulos para dibujar arcos
    angulo1 = np.arctan2(vec1_2d[1], vec1_2d[0])
    angulo2 = np.arctan2(vec2_2d[1], vec2_2d[0])
    angulo3 = np.arctan2(vec3_2d[1], vec3_2d[0])

    # Dibujar arcos para mostrar ángulos
    radio = 0.2 * min(np.linalg.norm(vec1_2d), np.linalg.norm(vec2_2d), np.linalg.norm(vec3_2d))

    t1 = np.linspace(min(angulo1, angulo2), max(angulo1, angulo2), 100)
    plt.plot(radio * np.cos(t1), radio * np.sin(t1), 'g-', lw=2, alpha=0.5)

    t2 = np.linspace(min(angulo1, angulo3), max(angulo1, angulo3), 100)
    plt.plot(radio * np.cos(t2), radio * np.sin(t2), 'r-', lw=2, alpha=0.5)

    plt.tight_layout()
    plt.show()

    # Explicación matemática usando los valores ya calculados
    print("\nFÓRMULA MATEMÁTICA:")
    print("  similitud_coseno(A, B) = (A·B) / (|A| × |B|)")
    print("Donde:")
    print("  • A·B es el producto punto de los vectores")
    print("  • |A| y |B| son las magnitudes (normas) de los vectores")
    print("\nEjemplo con valores reales de la matriz:")
    print(f"  • Similitud entre '{palabra_es}' y '{palabra_en_relacionada}': {sim_12:.4f}")
    print(f"  • Similitud entre '{palabra_es}' y '{palabra_en_no_relacionada}': {sim_13:.4f}")
    print("\nEsta representación visual usa PCA para reducir la dimensionalidad de los vectores,")
    print("preservando las similitudes calculadas en la matriz anterior.")

# ====================== FUNCIÓN PRINCIPAL ======================

def main():
    """Función principal del programa."""
    print("🛒 EJERCICIO DE SIMILITUD SEMÁNTICA: PRODUCTOS DE SUPERMERCADO 🛒")
    print("="*80)
    print("Este ejercicio demuestra el proceso completo de NLP:")
    print("1. Tokenización ✂️: Dividir el texto en unidades básicas")
    print("2. Normalización 🧹: Estandarizar el texto (minúsculas, lematización)")
    print("3. Vectorización 🔢: Convertir texto en representaciones numéricas")
    print("4. Similitud de coseno 📏: Medir la similitud semántica entre palabras en diferentes idiomas")
    print("="*80)

    # Definimos productos de supermercado en español e inglés
    productos_espanol = ["pan", "leche", "manzana", "cebolla", "jabón", "huevos", "queso"]
    productos_ingles = ["bread", "milk", "apple", "onion", "soap", "eggs", "cheese"]

    # Diccionario de traducciones correctas para verificación
    traducciones = {
        "pan": "bread",
        "leche": "milk",
        "manzana": "apple",
        "cebolla": "onion",
        "jabón": "soap",
        "huevos": "eggs",
        "queso": "cheese"
    }

    # Cargamos los modelos de lenguaje de spaCy
    print("\nCargando modelos de lenguaje...")
    try:
        nlp_es = spacy.load("es_core_news_md")
        print("✓ Modelo español cargado")
    except OSError:
        print("! Modelo español no encontrado. Instalando...")
        import subprocess
        subprocess.call(["python", "-m", "spacy", "download", "es_core_news_md"])
        nlp_es = spacy.load("es_core_news_md")

    try:
        nlp_en = spacy.load("en_core_web_md")
        print("✓ Modelo inglés cargado")
    except OSError:
        print("! Modelo inglés no encontrado. Instalando...")
        import subprocess
        subprocess.call(["python", "-m", "spacy", "download", "en_core_web_md"])
        nlp_en = spacy.load("en_core_web_md")

    # Demostramos la importancia de la normalización
    demostrar_mejora_normalizacion("Pan", "Bread", nlp_es, nlp_en)

    # Calculamos la similitud entre productos
    matriz_similitud, vectores_es, vectores_en = calcular_similitud_productos(
        productos_espanol,
        productos_ingles,
        traducciones,
        nlp_es,
        nlp_en
    )

    # Explicamos la similitud de coseno usando los valores de la matriz ya calculada
    explicar_similitud_coseno(matriz_similitud, productos_espanol, productos_ingles,
                             vectores_es, vectores_en, traducciones)

print("\n💡 CONCLUSIÓN")
print("Este ejercicio demuestra una limitación importante de los embeddings independientes por idioma:")
print("1. Los valores de similitud entre palabras equivalentes en diferentes idiomas")
print("   son extremadamente bajos (cercanos a 0) porque los espacios vectoriales no están alineados")
print("2. Para obtener similitudes semánticas significativas entre idiomas, se necesitarían:")
print("   - Embeddings multilingües específicamente entrenados (como MUSE, XLM-R o LaBSE)")
print("   - Técnicas de alineamiento de espacios vectoriales")
print("3. Los modelos de SpaCy estándar no están diseñados para comparaciones multilingües")
print("4. Esta limitación es crítica para aplicaciones como:")
print("   - Búsqueda multilingüe")
print("   - Sistemas de recomendación internacionales")
print("   - Traducción automática")

# ====================== EJECUCIÓN DEL PROGRAMA ======================

if __name__ == "__main__":
    main()

## Ejemplo de Embeddings Bilingues

In [ ]:
"""
EJERCICIO DE SIMILITUD SEMÁNTICA MULTILINGÜE MEJORADO: PRODUCTOS DE SUPERMERCADO 🛒

Este script demuestra el proceso de NLP para comparar productos de supermercado
en español e inglés usando embeddings multilingües, mostrando mejores resultados
en la captura de similitudes semánticas entre diferentes idiomas.
"""

# ====================== IMPORTACIÓN DE BIBLIOTECAS ======================
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# Instalamos sentence-transformers si es necesario reiniciar
# !pip install -q sentence-transformers

# Importamos SentenceTransformer para embeddings multilingües
from sentence_transformers import SentenceTransformer

# ====================== DEFINICIÓN DE FUNCIONES ======================

def procesar_palabra(palabra, modelo, mostrar_detalles=True):
    """
    Realiza el proceso de embedding para una palabra usando un modelo multilingüe:
    1. Normalización 🧹
    2. Vectorización con embedding multilingüe 🔢
    """
    # Normalizamos la palabra (minúsculas)
    palabra_norm = palabra.lower()

    # Generamos el embedding
    vector = modelo.encode(palabra_norm)

    if mostrar_detalles:
        print(f"\n🔍 Analizando: '{palabra}'")

        # PASO 1: NORMALIZACIÓN 🧹
        print(f"  🧹 Normalización:")
        print(f"    - Original: '{palabra}'")
        print(f"    - Minúsculas: '{palabra_norm}'")

        # PASO 2: VECTORIZACIÓN 🔢
        print(f"  🔢 Vectorización (embedding multilingüe):")
        print(f"    - Dimensiones del vector: {vector.shape}")
        print(f"    - Muestra del vector: {vector[:5]}...")

    # Devolvemos el vector procesado
    return vector

def calcular_similitud_productos(productos_es, productos_en, traducciones, modelo):
    """
    Calcula y visualiza la matriz de similitud entre productos en español e inglés.

    Args:
        productos_es: Lista de productos en español
        productos_en: Lista de productos en inglés
        traducciones: Diccionario con las traducciones correctas
        modelo: Modelo de embedding multilingüe
    """
    # Matrices para almacenar vectores y similitudes
    vectores_es = []
    vectores_en = []
    matriz_similitud = np.zeros((len(productos_es), len(productos_en)))

    # Procesamos cada producto en español
    print("\n🔄 PROCESANDO PRODUCTOS EN ESPAÑOL")
    for producto in productos_es:
        vector = procesar_palabra(producto, modelo, mostrar_detalles=True)
        vectores_es.append(vector)

    # Procesamos cada producto en inglés
    print("\n🔄 PROCESANDO PRODUCTOS EN INGLÉS")
    for producto in productos_en:
        vector = procesar_palabra(producto, modelo, mostrar_detalles=True)
        vectores_en.append(vector)

    # Calculamos la similitud de coseno entre cada par de productos
    print("\n📊 CALCULANDO MATRIZ DE SIMILITUD")
    for i, vec_es in enumerate(vectores_es):
        for j, vec_en in enumerate(vectores_en):
            # Preparamos los vectores para calcular similitud de coseno
            vec_es_reshaped = vec_es.reshape(1, -1)
            vec_en_reshaped = vec_en.reshape(1, -1)

            # FÓRMULA DE SIMILITUD DE COSENO: cos(θ) = (A·B)/(|A|·|B|)
            sim = cosine_similarity(vec_es_reshaped, vec_en_reshaped)[0][0]
            matriz_similitud[i, j] = sim

            # Destacamos si es la traducción correcta
            es_traduccion = productos_en[j] == traducciones[productos_es[i]]
            destacado = "✓" if es_traduccion else ""

            print(f"  Similitud entre '{productos_es[i]}' y '{productos_en[j]}': {sim:.4f} {destacado}")

    # Visualizamos la matriz de similitud
    visualizar_matriz_similitud(productos_es, productos_en, matriz_similitud, traducciones)

    # Encontramos las mejores coincidencias
    encontrar_mejores_coincidencias(productos_es, productos_en, matriz_similitud, traducciones)

    # Almacenar los resultados para usar en la visualización
    return matriz_similitud, vectores_es, vectores_en

def visualizar_matriz_similitud(productos_es, productos_en, matriz_similitud, traducciones):
    """
    Visualiza la matriz de similitud entre productos como un mapa de calor.
    """
    plt.figure(figsize=(10, 8))

    # Creamos el mapa de calor con etiquetas arriba
    ax = sns.heatmap(
        matriz_similitud,
        annot=True,
        fmt=".2f",
        cmap="Blues",
        xticklabels=productos_en,
        yticklabels=productos_es,
        vmin=-0.1,
        vmax=1,
        cbar_kws={'label': 'Similitud de Coseno'}
    )

    # Colocamos las etiquetas de columnas (productos ingleses) en la parte superior
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')

    # Añadimos un marcador para las traducciones correctas
    # Creamos un diccionario inverso para encontrar índices
    indices_en = {prod: i for i, prod in enumerate(productos_en)}

    for i, prod_es in enumerate(productos_es):
        traduccion = traducciones[prod_es]
        j = indices_en[traduccion]

        # Dibujamos un rectángulo alrededor de la traducción correcta
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False,
                                 edgecolor='red', lw=2, clip_on=False))

    plt.title("Similitud de Coseno entre Productos (Español - Inglés)", fontsize=14, pad=20)
    plt.xlabel("Productos en Inglés", fontsize=12)
    plt.ylabel("Productos en Español", fontsize=12)
    plt.tight_layout()
    plt.show()

def encontrar_mejores_coincidencias(productos_es, productos_en, matriz_similitud, traducciones):
    """
    Encuentra y muestra las mejores coincidencias entre productos.
    """
    print("\n🏆 MEJORES COINCIDENCIAS POR PRODUCTO")

    # Contamos cuántas veces acertamos la traducción correcta
    aciertos = 0

    for i, producto_es in enumerate(productos_es):
        # Encontramos el producto inglés con mayor similitud
        mejor_indice = np.argmax(matriz_similitud[i])
        mejor_similitud = matriz_similitud[i, mejor_indice]
        mejor_producto_en = productos_en[mejor_indice]

        # Verificamos si es la traducción correcta
        traduccion_correcta = traducciones[producto_es]
        es_correcta = mejor_producto_en == traduccion_correcta

        if es_correcta:
            aciertos += 1
            resultado = "✅ CORRECTO"
        else:
            resultado = f"❌ INCORRECTO (debería ser '{traduccion_correcta}')"

        print(f"  '{producto_es}' → '{mejor_producto_en}' (similitud: {mejor_similitud:.4f}) {resultado}")

        # Interpretamos la similitud
        if mejor_similitud > 0.7:
            interpretacion = "Alta similitud semántica"
        elif mejor_similitud > 0.4:
            interpretacion = "Similitud semántica moderada"
        elif mejor_similitud > 0.2:
            interpretacion = "Baja similitud semántica"
        else:
            interpretacion = "Similitud muy baja o nula"

        print(f"    Interpretación: {interpretacion}")

    # Mostramos la precisión general del modelo
    precision = aciertos / len(productos_es) * 100
    print(f"\n📈 Precisión del modelo: {precision:.1f}% ({aciertos}/{len(productos_es)} aciertos)")

def demostrar_mejora_modelos(palabra_es, palabra_en, modelo):
    """
    Demuestra la mejora usando embeddings multilingües.
    """
    print("\n🧪 DEMOSTRACIÓN: MEJORA CON EMBEDDINGS MULTILINGÜES")

    # Generamos embeddings con el modelo multilingüe
    vector_es = modelo.encode(palabra_es.lower())
    vector_en = modelo.encode(palabra_en.lower())

    # Calculamos similitud
    sim = cosine_similarity(
        vector_es.reshape(1, -1),
        vector_en.reshape(1, -1)
    )[0][0]

    print(f"Comparando '{palabra_es}' con '{palabra_en}' usando LaBSE:")
    print(f"  • Similitud: {sim:.4f}")

    if sim > 0.8:
        interpretacion = "Alta similitud (excelente alineación multilingüe)"
    elif sim > 0.6:
        interpretacion = "Buena similitud (alineación efectiva)"
    elif sim > 0.4:
        interpretacion = "Similitud moderada"
    else:
        interpretacion = "Baja similitud (posible falta de alineación)"

    print(f"  • Interpretación: {interpretacion}")
    print(f"  • Los modelos multilingües como LaBSE están específicamente entrenados")
    print(f"    para alinear términos equivalentes en diferentes idiomas.")

def explicar_similitud_coseno(modelo, productos_es, productos_en, traducciones):
    """
    Explica el concepto de similitud de coseno con una visualización usando vectores reales.

    Args:
        modelo: Modelo de embedding multilingüe
        productos_es: Lista de productos en español
        productos_en: Lista de productos en inglés
        traducciones: Diccionario con las traducciones correctas
    """
    print("\n📐 EXPLICACIÓN: SIMILITUD DE COSENO CON EMBEDDINGS MULTILINGÜES")
    print("La similitud de coseno mide el ángulo entre dos vectores:")
    print("  • Similitud = 1: Los vectores apuntan en la misma dirección (productos idénticos)")
    print("  • Similitud = 0: Los vectores son perpendiculares (productos no relacionados)")
    print("  • Similitud = -1: Los vectores apuntan en direcciones opuestas (raramente ocurre)")

    # Obtener vectores reales de los modelos
    producto_es = "pan"  # Producto en español para demostración
    producto_en_correcto = traducciones[producto_es]  # Traducción correcta
    producto_en_no_relacionado = "soap"  # Producto no relacionado

    vec_es = modelo.encode(producto_es.lower())
    vec_en_correcto = modelo.encode(producto_en_correcto.lower())
    vec_en_no_rel = modelo.encode(producto_en_no_relacionado.lower())

    # Calcular similitudes reales usando los vectores completos
    sim_correcta = cosine_similarity(
        vec_es.reshape(1, -1),
        vec_en_correcto.reshape(1, -1)
    )[0][0]

    sim_no_rel = cosine_similarity(
        vec_es.reshape(1, -1),
        vec_en_no_rel.reshape(1, -1)
    )[0][0]

    # Reducir dimensionalidad para visualización (PCA a 2D)
    pca = PCA(n_components=2)

    # Juntar los vectores para aplicar PCA
    vectores_combinados = np.vstack([vec_es, vec_en_correcto, vec_en_no_rel])
    vectores_2d = pca.fit_transform(vectores_combinados)

    # Extraer los vectores reducidos
    vec1_2d = vectores_2d[0]  # Pan
    vec2_2d = vectores_2d[1]  # Bread
    vec3_2d = vectores_2d[2]  # Soap

    # Crear visualización
    plt.figure(figsize=(8, 6))
    origen = [0, 0]

    # Dibujamos los vectores en 2D
    plt.quiver(*origen, *vec1_2d, angles='xy', scale_units='xy', scale=1, color='blue',
               label=f'{producto_es} (español)')
    plt.quiver(*origen, *vec2_2d, angles='xy', scale_units='xy', scale=1, color='green',
               label=f'{producto_en_correcto} (inglés) - sim:{sim_correcta:.2f}')
    plt.quiver(*origen, *vec3_2d, angles='xy', scale_units='xy', scale=1, color='red',
               label=f'{producto_en_no_relacionado} (inglés) - sim:{sim_no_rel:.2f}')

    # Configurar gráfico
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    plt.legend(loc='upper left')
    plt.title('Similitud de Coseno: Representación Geométrica (embeddings multilingües)')

    # Calcular ángulos para dibujar arcos
    angulo1 = np.arctan2(vec1_2d[1], vec1_2d[0])
    angulo2 = np.arctan2(vec2_2d[1], vec2_2d[0])
    angulo3 = np.arctan2(vec3_2d[1], vec3_2d[0])

    # Dibujar arcos para mostrar ángulos
    radio = 0.2 * min(np.linalg.norm(vec1_2d), np.linalg.norm(vec2_2d), np.linalg.norm(vec3_2d))

    t1 = np.linspace(min(angulo1, angulo2), max(angulo1, angulo2), 100)
    plt.plot(radio * np.cos(t1), radio * np.sin(t1), 'g-', lw=2, alpha=0.5)

    t2 = np.linspace(min(angulo1, angulo3), max(angulo1, angulo3), 100)
    plt.plot(radio * np.cos(t2), radio * np.sin(t2), 'r-', lw=2, alpha=0.5)

    plt.tight_layout()
    plt.show()

    # Explicación matemática usando los vectores reales
    print("\nFÓRMULA MATEMÁTICA:")
    print("  similitud_coseno(A, B) = (A·B) / (|A| × |B|)")
    print("Donde:")
    print("  • A·B es el producto punto de los vectores")
    print("  • |A| y |B| son las magnitudes (normas) de los vectores")
    print("\nEjemplo con vectores reales:")
    print(f"  • Producto punto de '{producto_es}' y '{producto_en_correcto}': {np.dot(vec_es, vec_en_correcto):.4f}")
    print(f"  • Magnitud de '{producto_es}': {np.linalg.norm(vec_es):.4f}")
    print(f"  • Magnitud de '{producto_en_correcto}': {np.linalg.norm(vec_en_correcto):.4f}")
    print(f"  • Similitud de coseno: {sim_correcta:.4f}")
    print(f"\nComparación con vector no relacionado:")
    print(f"  • Similitud entre '{producto_es}' y '{producto_en_no_relacionado}': {sim_no_rel:.4f}")

# ====================== FUNCIÓN PRINCIPAL ======================

def main():
    """Función principal del programa."""
    print("🛒 EJERCICIO DE SIMILITUD SEMÁNTICA MULTILINGÜE: PRODUCTOS DE SUPERMERCADO 🛒")
    print("="*80)
    print("Este ejercicio demuestra el proceso de NLP usando embeddings multilingües:")
    print("1. Normalización 🧹: Estandarizar el texto (minúsculas)")
    print("2. Vectorización multilingüe 🔢: Convertir texto a representaciones vectoriales alineadas")
    print("3. Similitud de coseno 📏: Medir la similitud semántica entre palabras en diferentes idiomas")
    print("="*80)

    # Definimos productos de supermercado en español e inglés
    productos_espanol = ["pan", "leche", "manzana", "cebolla", "jabón", "huevos", "queso"]
    productos_ingles = ["bread", "milk", "apple", "onion", "soap", "eggs", "cheese"]

    # Diccionario de traducciones correctas para verificación
    traducciones = {
        "pan": "bread",
        "leche": "milk",
        "manzana": "apple",
        "cebolla": "onion",
        "jabón": "soap",
        "huevos": "eggs",
        "queso": "cheese"
    }

    # Cargamos el modelo de embedding multilingüe
    print("\nCargando modelo de embedding multilingüe...")
    try:
        # Cargamos el modelo LaBSE (Language-agnostic BERT Sentence Encoder)
        # Este modelo está específicamente diseñado para similitud semántica entre idiomas
        modelo = SentenceTransformer('sentence-transformers/LaBSE')
        print("✓ Modelo LaBSE cargado")
    except Exception as e:
        print(f"! Error al cargar el modelo: {e}")
        print("Asegúrate de tener instalada la biblioteca 'sentence-transformers':")
        print("!pip install -q sentence-transformers")
        return

    # Demostramos la mejora usando embeddings multilingües
    demostrar_mejora_modelos("Pan", "Bread", modelo)

    # Calculamos la similitud entre productos
    matriz_similitud, vectores_es, vectores_en = calcular_similitud_productos(
        productos_espanol,
        productos_ingles,
        traducciones,
        modelo
    )

    # Explicamos la similitud de coseno usando vectores reales
    explicar_similitud_coseno(modelo, productos_espanol, productos_ingles, traducciones)

    print("\n💡 CONCLUSIÓN")
    print("Este ejercicio muestra cómo los embeddings multilingües mejoran significativamente")
    print("la detección de equivalencias semánticas entre palabras de diferentes idiomas.")
    print("Observaciones importantes:")
    print("1. Los embeddings multilingües (LaBSE) están específicamente entrenados para")
    print("   alinear palabras con el mismo significado en diferentes idiomas.")
    print("2. Esto se refleja en valores de similitud mucho más altos para traducciones correctas.")
    print("3. La matriz de similitud ahora muestra una clara diagonal de valores altos, lo que")
    print("   indica una correcta identificación de productos equivalentes.")
    print("4. Esta tecnología es fundamental en aplicaciones modernas de PLN como:")
    print("   - Búsqueda y recuperación de información multilingüe")
    print("   - Sistemas de recomendación internacionales")
    print("   - Traducción automática de alta calidad")
    print("   - Asistentes virtuales multilingües")

# ====================== EJECUCIÓN DEL PROGRAMA ======================

if __name__ == "__main__":
    main()